In [1]:
import aquamonitor as am

# Query project-specific water chemistry data using the Aquamonitor API

AquaMonitor-Python is a Python library providing external access to Nivabasen, NIVA's internal water chemistry and ecology database.

## 1. Login

There are several login options:

 1. **Create a file named `.auth`** in the same folder as `AquaMonitor.py` with the format shown below. `Aquamonitor-Python` will then attempt to read your credentials from this file automatically whenever they are required. This is the easiest solution on secure systems, but **note that your username and password will be stored as plain text, so use with caution**.
 
            [Auth]
            username = YourUsername
            password = YourPassword
            
            
 2. **Use the `login()` function**, as shown below. This will create an access token that is valid for one day, which you can then pass to any subsequent queries (see below for examples)
 
 3. **Do nothing**. If you do not provide an access token and the `.auth` file cannot be found, you will be prompted to enter your username and password. Note that you will have to do this for *every* query, so in most cases one of the other options will be preferred    

In [2]:
# Get access token
token = am.login()

Please enter your credentials.


Username:  ···
Password:  ··············


## 2. List projects

The `get_projects` function returns a list of all projects in the database. You can search this dataframe to identify the correct project ID for the project you are interested in.

In [3]:
proj_df = am.get_projects(token=token)
print(f"{len(proj_df)} projects in the database.")
proj_df.head()

1334 projects in the database.


,project_id,project_code,project_name,description
0,1,NaN,xxx,NaN
1,55,NaN,PARCOM_Aquateam,NaN
2,56,NaN,PARCOM_NIVA,NaN
3,58,NaN,MAR_BOK,Fagdata
4,59,NaN,MAR_IMA,Fagdata


In [4]:
# Find the project code for the 2019 '1000 Lakes' survey
proj_df.query("project_name.str.contains('Nasjonal')", engine="python")

,project_id,project_code,project_name,description
1200,12433,190091,Nasjonal Innsjøundersøkelse 2019,1000-sjøer undersøkelse gjennomført i 2019


## 3. Get stations in project

Use `get_project_stations` to get a list of all stations associated with your project of interest.

**To do:** It would be great if this API endpoint could also return the geographic co-ordinates (lat/lon) for each station. At present, I think this can only be achieved by iterating over the `api/projects/{proj_id}/stations/{stn_id}/samplePoint` endpoint for each station, which is slow.

In [5]:
# Project of interest
proj_id = 12433

stn_df = am.get_project_stations(proj_id, token=token)
print(f"{len(stn_df)} stations in project.")
stn_df.head()

1003 stations in project.


,project_id,station_id,station_code,station_name,type
0,12433,26070,221-1-2,Langtjern,Innsjø
1,12433,26071,101-2-7,Hokksjøen,Innsjø
2,12433,26072,402-2-13,Sætertjern,Innsjø
3,12433,26073,419-1-25,Mjøgsjøen,Innsjø
4,12433,26074,425-2-2,Kottern,Innsjø


## 4. Get water chemistry

Use `get_project_chemistry` to extract all the water chemistry data for your project and time period of of interest.

In [6]:
%%time

# Period of interest
st_dt = "01.01.2019"
end_dt = "31.12.2019"

df = am.get_project_chemistry(proj_id, st_dt, end_dt, token=token)
df.head()

CPU times: user 5.53 s, sys: 221 ms, total: 5.75 s
Wall time: 30.8 s


,project_id,project_name,station_id,station_code,station_name,sample_date,depth1,depth2,parameter_name,flag,value,unit
0,12433,Nasjonal Innsjøundersøkelse 2019,26070,221-1-2,Langtjern,2019-10-25 00:00:00+00:00,0.0,0.0,Al,NaN,280.000,µg/l
1,12433,Nasjonal Innsjøundersøkelse 2019,26070,221-1-2,Langtjern,2019-10-25 00:00:00+00:00,0.0,0.0,Al/IL,NaN,130.000,µg/l
2,12433,Nasjonal Innsjøundersøkelse 2019,26070,221-1-2,Langtjern,2019-10-25 00:00:00+00:00,0.0,0.0,Al/L,NaN,20.000,µg/l
3,12433,Nasjonal Innsjøundersøkelse 2019,26070,221-1-2,Langtjern,2019-10-25 00:00:00+00:00,0.0,0.0,Al/R,NaN,150.000,µg/l
4,12433,Nasjonal Innsjøundersøkelse 2019,26070,221-1-2,Langtjern,2019-10-25 00:00:00+00:00,0.0,0.0,Alk_4.5,NaN,0.054,mmol/l
